In [ ]:
from google.colab import files
uploaded = files.upload()



Saving cases_deaths.csv to cases_deaths.csv


In [8]:
from google.colab import files
uploaded = files.upload()


Saving ne_110m_admin_0_countries.cpg to ne_110m_admin_0_countries.cpg
Saving ne_110m_admin_0_countries.dbf to ne_110m_admin_0_countries.dbf
Saving ne_110m_admin_0_countries.prj to ne_110m_admin_0_countries.prj
Saving ne_110m_admin_0_countries.README.html to ne_110m_admin_0_countries.README.html
Saving ne_110m_admin_0_countries.shp to ne_110m_admin_0_countries.shp
Saving ne_110m_admin_0_countries.shx to ne_110m_admin_0_countries.shx
Saving ne_110m_admin_0_countries.VERSION.txt to ne_110m_admin_0_countries.VERSION.txt


In [9]:
import geopandas as gpd

world = gpd.read_file("ne_110m_admin_0_countries.shp")
world = world[(world.POP_EST > 0) & (world.NAME != "Antarctica")]
world.head()


,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,1,6,Fiji,FJI,0,2,Sovereign country,1,Fiji,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((180 -16.06713, 180 -16.55522, ..."
1,Admin-0 country,1,3,United Republic of Tanzania,TZA,0,2,Sovereign country,1,United Republic of Tanzania,...,None,None,None,None,None,None,None,None,None,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3..."
2,Admin-0 country,1,7,Western Sahara,SAH,0,2,Indeterminate,1,Western Sahara,...,Unrecognized,Unrecognized,Unrecognized,None,None,Unrecognized,None,None,None,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Admin-0 country,1,2,Canada,CAN,0,2,Sovereign country,1,Canada,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-122.84 49, -122.97421 49.0025..."
4,Admin-0 country,1,2,United States of America,US1,1,2,Country,1,United States of America,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-122.84 49, -120 49, -117.0312..."


In [12]:
df_clean = df[["country", "date", "total_deaths_per_million"]].copy()
df_clean["date"] = pd.to_datetime(df_clean["date"])

# Drop missing values
df_clean = df_clean.dropna(subset=["total_deaths_per_million"])

df_clean.head()


,country,date,total_deaths_per_million
0,Afghanistan,2020-01-04,0.0
1,Afghanistan,2020-01-05,0.0
2,Afghanistan,2020-01-06,0.0
3,Afghanistan,2020-01-07,0.0
4,Afghanistan,2020-01-08,0.0


In [28]:
import pandas as pd, numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


df_use = df[["country","date","total_deaths","total_deaths_per_million"]].copy()
df_use["date"] = pd.to_datetime(df_use["date"])
df_use = df_use.sort_values(["country","date"])


def monthly_ffill(g):
    g = g.set_index("date").resample("M").last()
    return g.ffill()

df_m = (df_use.groupby("country", group_keys=False)
               .apply(monthly_ffill)
               .reset_index())

df_m["month"] = df_m["date"].dt.strftime("%Y-%m")




months = sorted(df_m["month"].unique())
zmax = float(df_m["total_deaths_per_million"].max())  # global color range
bar_xmax = float(df_m["total_deaths_per_million"].max()) * 1.05  # lock bar axis

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type":"choropleth"}, {"type":"bar"}]],
    column_widths=[0.70, 0.30],
    horizontal_spacing=0.04,
    subplot_titles=("Deaths per Million (choropleth)", "Top 10 by deaths/million")
)


m0 = months[0]
d0 = df_m[df_m["month"]==m0]

map0 = go.Choropleth(
    locations=d0["country"],
    locationmode="country names",
    z=d0["total_deaths_per_million"],
    text=d0["country"],
    colorscale="Reds",
    zmin=0, zmax=zmax,
    colorbar_title="Deaths / million",
    customdata=d0["total_deaths"],
    hovertemplate="<b>%{text}</b><br>Deaths/million: %{z:.0f}<br>Total deaths: %{customdata:,d}<extra></extra>"
)
fig.add_trace(map0, 1, 1)

top0 = (d0.dropna(subset=["total_deaths_per_million"])
          .nlargest(10, "total_deaths_per_million")
          .iloc[::-1])
bar0 = go.Bar(
    x=top0["total_deaths_per_million"],
    y=top0["country"],
    orientation="h",
    marker_color="#b30000",
    hovertemplate="<b>%{y}</b><br>Deaths/million: %{x:,.0f}<extra></extra>"
)
fig.add_trace(bar0, 1, 2)


frames = []
for m in months:
    dm = df_m[df_m["month"]==m]
    map_f = go.Choropleth(
        locations=dm["country"],
        locationmode="country names",
        z=dm["total_deaths_per_million"],
        text=dm["country"],
        colorscale="Reds",
        zmin=0, zmax=zmax,
        customdata=dm["total_deaths"],
        hovertemplate="<b>%{text}</b><br>Deaths/million: %{z:.0f}<br>Total deaths: %{customdata:,d}<extra></extra>",
        showscale=False
    )
    top = (dm.dropna(subset=["total_deaths_per_million"])
             .nlargest(10, "total_deaths_per_million")
             .iloc[::-1])
    bar_f = go.Bar(
        x=top["total_deaths_per_million"],
        y=top["country"],
        orientation="h",
        marker_color="#b30000",
        hovertemplate="<b>%{y}</b><br>Deaths/million: %{x:,.0f}<extra></extra>"
    )
    frames.append(go.Frame(data=[map_f, bar_f], name=m))

fig.frames = frames

fig.update_layout(
    title="COVID-19 Cumulative Deaths per Million — Map + Top-10 (Animated)",
    geo=dict(showframe=False, showcoastlines=True),
    updatemenus=[{
        "type":"buttons",
        "x":0.02, "y":-0.08,
        "buttons":[
            {"label":"▶ Play", "method":"animate",
             "args":[None, {"fromcurrent":True, "frame":{"duration":400, "redraw":True}, "transition":{"duration":0}}]},
            {"label":"⏸ Pause", "method":"animate",
             "args":[[None], {"frame":{"duration":0, "redraw":False}, "mode":"immediate"}]}
        ]
    }],
    sliders=[{
        "active":0, "x":0.12, "y":-0.06, "len":0.75,
        "steps":[{"label":m, "method":"animate",
                  "args":[[m], {"mode":"immediate","frame":{"duration":0,"redraw":True}}]} for m in months]
    }],
    margin=dict(l=40, r=20, t=60, b=80)
)

# Lock the bar axis across all frames & pretty ticks
fig.update_xaxes(row=1, col=2, title="Deaths per million", range=[0, bar_xmax], tickformat=",.0f")
fig.update_yaxes(row=1, col=2, title=None)

fig.show()


/tmp/ipython-input-2371990964.py:12: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

/tmp/ipython-input-2371990964.py:16: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [29]:
OUT = "covid_deaths_world_with_top10.html"
fig.write_html(OUT)
print("Saved:", OUT)

from google.colab import files
files.download(OUT)


Saved: covid_deaths_world_with_top10.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>